In [1]:
from collections import defaultdict
from operator import itemgetter
from time import time

from binance.client import Client

In [2]:
FEE = 0.000000005
PRIMARY = ['BTC', 'ETH', 'BETH', 'USDT']

In [3]:
def main():
    start_time = time()

    prices = get_prices()
    prices_time = time()
    print(f"Downloaded in: {prices_time - start_time:.4f}s")

    triangles = list(find_triangles(prices))
    print(f"Computed in: {time() - prices_time:.4f}s")

    if triangles:
        for triangle in sorted(triangles, key=itemgetter('profit'), reverse=True):
            describe_triangle(prices, triangle)
    else:
        print("No triangles found, try again!")

In [4]:
def get_prices():
    client = Client(None, None)
    prices = client.get_orderbook_tickers()
    prepared = defaultdict(dict)
    for ticker in prices:
        pair = ticker['symbol']
        ask = float(ticker['askPrice'])
        bid = float(ticker['bidPrice'])
        if ask == 0.0:
            continue
        for primary in PRIMARY:
            if pair.endswith(primary):
                secondary = pair[:-len(primary)]
                prepared[primary][secondary] = 1 / ask
                prepared[secondary][primary] = bid
    return prepared

In [5]:
def find_triangles(prices):
    triangles = []
    starting_coin = 'ETH'
    for triangle in recurse_triangle(prices, starting_coin, starting_coin):
        coins = set(triangle['coins'])
        if not any(prev_triangle == coins for prev_triangle in triangles):
            yield triangle
            triangles.append(coins)

In [6]:
def recurse_triangle(prices, current_coin, starting_coin, depth_left=3, amount=1.0):
    if depth_left > 0:
        pairs = prices[current_coin]
        for coin, price in pairs.items():
            new_price = (amount * price) * (1.0 - FEE)
            for triangle in recurse_triangle(prices, coin, starting_coin, depth_left - 1, new_price):
                triangle['coins'] = triangle['coins'] + [current_coin]
                yield triangle
    elif current_coin == starting_coin and amount > 1.0:
        yield {
            'coins': [current_coin],
            'profit': amount
        }

In [7]:
def describe_triangle(prices, triangle):
    coins = triangle['coins']
    price_percentage = (triangle['profit'] - 1.0) * 100
    print(f"{'->'.join(coins):26} {round(price_percentage, 4):-7}% <- profit!")
    for i in range(len(coins) - 1):
        first = coins[i]
        second = coins[i + 1]
        print(f"     {second:4} / {first:4}: {prices[first][second]:-17.8f}")
    print('')

In [8]:
if __name__ == '__main__':
    main()

Downloaded in: 0.9750s
Computed in: 0.0143s
ETH->DASH->USDT->ETH        0.0259% <- profit!
     DASH / ETH :       17.31601732
     USDT / DASH:      189.27000000
     ETH  / USDT:        0.00030450

ETH->USDT->NANO->ETH        0.0207% <- profit!
     USDT / ETH :     3284.11000000
     NANO / USDT:        0.14970956
     ETH  / NANO:        0.00202700

ETH->USDT->BNB->ETH         0.0139% <- profit!
     USDT / ETH :     3284.11000000
     BNB  / USDT:        0.00247255
     ETH  / BNB :        0.12306000

